# Stock price prediction: a machine learning approach

Purpose: Predict stock price using Machine Learning methods, such as Support Vector Machines, Deep Neural Networks and Random Forest
Authors: Caio Lopes De Souza, Silvio Sandoval Zocchi, Gabriel Rodrigues Palma

## Packages used in this project

In [1]:
# Data extraction modules
!pip install investpy 
import investpy

# Data manipulation modules
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import yfinance as yf

# Data visualization modules
import matplotlib.pyplot as plt

# Machine Learning modules
from keras.models import Sequential
from keras.layers import Activation, Dense
from tensorflow.keras.metrics import Mean
# Cross-validation modules 
from sklearn.model_selection import TimeSeriesSplit

# Grid search module
from sklearn.model_selection import GridSearchCV

# Evaluation metrics modules
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from tensorflow.keras.metrics import Precision

## Functions used in this project

In [2]:
def get_train_test_data(cripto_series, test_percentage):
    '''This functions creates the training and test data for the stock time series'''
    time_series_length = len(cripto_series)
    test_series = cripto_series.iloc[int(time_series_length*(1-test_percentage)):time_series_length]
    train_series = cripto_series.iloc[:-int(time_series_length*test_percentage)]
    
    return(test_series, train_series)

def get_rate_of_change(cripto_series, variable, n_days):
    '''This function computes several values for the rate of change and add them into a dataset'''
    for day in n_days: 
        variable_name = f"{day}_days_of_change"
        cripto_series[variable_name] = (cripto_series[variable] - cripto_series[variable].shift(day)) / cripto_series[variable].shift(day)
        
def get_moving_average(cripto_series, variable, n_days):
    '''This function computes several values for the moving average and add them into a dataset'''
    for day in n_days:
        variable_name = f"{day}_day_moving_average"
        cripto_series[variable_name] = cripto_series[variable].rolling(day).mean()
        
def get_ratio(cripto_series, variable_1, variable_2):
    '''This function computes the ratio between two variables and add it into a dataset'''
    variable_name = f"{variable_1}_{variable_2}_ratio"
    cripto_series[variable_name] = cripto_series[variable_1] / cripto_series[variable_2]
    
def get_difference(cripto_series, variable_1, variable_2):
    '''This function computes the difference between the values of two variables and add it into a dataset'''
    variable_name = f"{variable_1}_{variable_2}_difference"
    cripto_series[variable_name] = cripto_series[variable_1] - cripto_series[variable_2] 
    
def train_and_predict(model, train_predictors, train_target, test_predictors):
    '''This function fits a model and then make predictions on it for a given amount of data'''
    model.fit(train_predictors, train_target)
    preds = model.predict(test_predictors)
    preds = pd.Series(preds, index=test_series_2.index)
    return preds

## Reading the data

In [3]:
stock_series = investpy.stocks.get_stock_historical_data('PETR4', 'brazil', from_date = '01/01/2002', to_date='01/08/2022')

In [4]:
stock_series.head()

,Open,High,Low,Close,Volume,Currency
Date,,,,,,
2002-01-02,4.73,4.79,4.72,4.73,23979552,BRL
2002-01-03,4.75,4.80,4.74,4.76,39124800,BRL
2002-01-04,4.76,4.76,4.67,4.73,21548832,BRL
2002-01-07,4.71,4.87,4.71,4.83,40547104,BRL
2002-01-08,4.83,4.83,4.77,4.80,34469888,BRL


## Train and test data

In [5]:
test_series, train_series = get_train_test_data(stock_series, test_percentage=0.1)

In [6]:
train_series['target']  = np.array(train_series['Close'].shift(-1) > train_series['Close']).astype(int) 

## Creating new variables

In [12]:
get_rate_of_change(train_series, 'Close', [3, 5, 7, 15])

In [13]:
get_difference(train_series, 'Close', 'Open')
get_difference(train_series, 'High', 'Low')

In [14]:
train_series['low_dif'] = (train_series['Low'] - train_series['Low'].shift(1))
train_series['high_dif'] = train_series['High'] - train_series['High'].shift(1)

In [15]:
get_ratio(train_series, 'Open', 'Close')
get_ratio(train_series, 'High', 'Close')
get_ratio(train_series, 'Low', 'Close')

In [16]:
get_moving_average(train_series, 'Close', [3, 5, 7, 15])

In [17]:
train_series.drop('Currency', axis=1, inplace=True)

In [18]:
train_series.dropna(axis=0, inplace=True)

In [19]:
test_series['target']  = np.array(test_series['Close'].shift(-1) > test_series['Close']).astype(int) 

In [20]:
get_rate_of_change(test_series, 'Close', [3, 5, 7, 15])

In [21]:
get_difference(test_series, 'Close', 'Open')
get_difference(test_series, 'High', 'Low')

In [22]:
test_series['low_dif'] = (test_series['Low'] - test_series['Low'].shift(1))
test_series['high_dif'] = test_series['High'] - test_series['High'].shift(1)

In [23]:
get_ratio(test_series, 'Open', 'Close')
get_ratio(test_series, 'High', 'Close')
get_ratio(test_series, 'Low', 'Close')

In [24]:
get_moving_average(test_series, 'Close', [3, 5, 7, 15])

In [25]:
test_series.drop('Currency', axis=1, inplace=True)

In [26]:
test_series.dropna(axis=0, inplace=True)

## Model Approach

In [29]:
train_series.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'target', '3_days_of_change',
       '5_days_of_change', '7_days_of_change', '15_days_of_change',
       'Close_Open_difference', 'High_Low_difference', 'low_dif', 'high_dif',
       'Open_Close_ratio', 'High_Close_ratio', 'Low_Close_ratio',
       '3_day_moving_average', '5_day_moving_average', '7_day_moving_average',
       '15_day_moving_average'],
      dtype='object')

In [30]:
predictors = ['Open', 'High', 'Low', 'Close', 'Volume', 'target', '3_days_of_change',
       '5_days_of_change', '7_days_of_change', '15_days_of_change',
       'Close_Open_difference', 'High_Low_difference', 'low_dif', 'high_dif',
       'Open_Close_ratio', 'High_Close_ratio', 'Low_Close_ratio',
       '3_day_moving_average', '5_day_moving_average', '7_day_moving_average',
       '15_day_moving_average']

In [31]:
len(predictors)

21

In [222]:
tscv = TimeSeriesSplit()
for train, test in tscv.split(train_series):
    print(len(train), len(test))

763 762
1525 762
2287 762
3049 762
3811 762


In [62]:
tscv = TimeSeriesSplit()
for train_index, test_index in tscv.split(train_series):
    train_series[:len(train_index)], test_series[:len(test_index)]

In [97]:
def create_dnn(n_neurons, n_layers, activation_function, train_data):
    '''This function creates a DNN architeture  from the number of neurons, the number of layers and a activation function'''
    model = Sequential()
    for layer in range(n_layers):
        if n_layers <= len(n_neurons):
            model.add(Dense(units=n_neurons[layer], activation=activation_function, input_shape=(train_data[predictors].shape[1:])))
    return model

In [85]:
model = create_dnn([5, 3, 2, 1], 3, 'relu', train_series)

In [86]:
model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.

In [98]:
def model_evaluate(model, train_data, test_data, target):
    '''This function fits and return the precision score for a model'''
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[Precision()])
    model.fit(train_data[predictors], train_data[target], batch_size=10, epochs=100)
    preds = model.predict(test_series[predictors])
    preds = preds.reshape(-1)
    preds = pd.Series(preds, index=test_series.index)
    return precision_score(test_series[target], preds)

In [99]:
def mean_of_list(list):
    '''This function computes the mean for a list'''
    return sum(list) / len(list)

In [100]:
def cross_get_cv_metrics(activation_list, n_neurons_list, n_layers_list, train_data, test_data, target):
    '''This function makes a grid search for a set of chosen hyperparameters'''
    dictionary = {}
    results_grid_search = []
    parameters = []
    results_cv = []
    split = TimeSeriesSplit()
    for activation in activation_list:
        for neuron in n_neurons_list:
            for layer in n_layers_list:
                model = create_dnn(n_neurons = neuron, n_layers = layer, activation_function = activation, train_data = train_series)
                parameters.append([activation, neuron, layer])
                for train_index, test_index in split.split(train_series):
                    precision = model_evaluate(model, train_data[:len(train_index)], test_data[:len(test_index)], target)
                    results_cv.append(precision)
                mean_cv = mean_of_list(results_cv)
                results_grid_search.append(mean_cv)
                dictionary[mean_cv] = parameters
    return dictionary.get(max(dictionary))

In [101]:
cross_get_cv_metrics(['relu', 'sigmoid'], 
                     n_neurons_list = [[32, 16, 10, 6, 1], [64, 32, 16, 10, 6, 1], [64, 42, 26, 12, 6, 1]], 
                     n_layers_list = [5, 6], 
                     train_data=train_series, test_data=test_series, target='target')

Epoch 1/100
77/77 [==============================] - 1s 2ms/step - loss: 7.8237 - precision_36: 0.0000e+00
Epoch 2/100
77/77 [==============================] - 0s 3ms/step - loss: 7.8237 - precision_36: 0.0000e+00
Epoch 3/100
77/77 [==============================] - 0s 2ms/step - loss: 7.8237 - precision_36: 0.0000e+00
Epoch 4/100
77/77 [==============================] - 0s 3ms/step - loss: 7.8237 - precision_36: 0.0000e+00
Epoch 5/100
77/77 [==============================] - 0s 2ms/step - loss: 7.8237 - precision_36: 0.0000e+00
Epoch 6/100
77/77 [==============================] - 0s 2ms/step - loss: 7.8237 - precision_36: 0.0000e+00
Epoch 7/100
77/77 [==============================] - 0s 2ms/step - loss: 7.8237 - precision_36: 0.0000e+00
Epoch 8/100
77/77 [==============================] - 0s 2ms/step - loss: 7.8237 - precision_36: 0.0000e+00
Epoch 9/100
77/77 [==============================] - 0s 2ms/step - loss: 7.8237 - precision_36: 0.0000e+00
Epoch 10/100
77/77 [=================

77/77 [==============================] - 0s 3ms/step - loss: 7.8237 - precision_36: 0.0000e+00
Epoch 77/100
77/77 [==============================] - 0s 2ms/step - loss: 7.8237 - precision_36: 0.0000e+00
Epoch 78/100
77/77 [==============================] - 0s 2ms/step - loss: 7.8237 - precision_36: 0.0000e+00
Epoch 79/100
77/77 [==============================] - 0s 2ms/step - loss: 7.8237 - precision_36: 0.0000e+00
Epoch 80/100
77/77 [==============================] - 0s 2ms/step - loss: 7.8237 - precision_36: 0.0000e+00
Epoch 81/100
77/77 [==============================] - 0s 2ms/step - loss: 7.8237 - precision_36: 0.0000e+00
Epoch 82/100
77/77 [==============================] - 0s 2ms/step - loss: 7.8237 - precision_36: 0.0000e+00
Epoch 83/100
77/77 [==============================] - 0s 2ms/step - loss: 7.8237 - precision_36: 0.0000e+00
Epoch 84/100
77/77 [==============================] - 0s 3ms/step - loss: 7.8237 - precision_36: 0.0000e+00
Epoch 85/100
77/77 [=====================

c:\users\caiol\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


153/153 [==============================] - 1s 3ms/step - loss: 8.0311 - precision_37: 0.0000e+00
Epoch 2/100
153/153 [==============================] - 0s 2ms/step - loss: 8.0311 - precision_37: 0.0000e+00
Epoch 3/100
153/153 [==============================] - 0s 2ms/step - loss: 8.0311 - precision_37: 0.0000e+00
Epoch 4/100
153/153 [==============================] - 0s 2ms/step - loss: 8.0311 - precision_37: 0.0000e+00
Epoch 5/100
153/153 [==============================] - 0s 2ms/step - loss: 8.0311 - precision_37: 0.0000e+00
Epoch 6/100
153/153 [==============================] - 0s 2ms/step - loss: 8.0311 - precision_37: 0.0000e+00
Epoch 7/100
153/153 [==============================] - 0s 2ms/step - loss: 8.0311 - precision_37: 0.0000e+00
Epoch 8/100
153/153 [==============================] - 0s 2ms/step - loss: 8.0311 - precision_37: 0.0000e+00
Epoch 9/100
153/153 [==============================] - 0s 2ms/step - loss: 8.0311 - precision_37: 0.0000e+00
Epoch 10/100
153/153 [=========

153/153 [==============================] - 0s 2ms/step - loss: 8.0311 - precision_37: 0.0000e+00
Epoch 76/100
153/153 [==============================] - 0s 2ms/step - loss: 8.0311 - precision_37: 0.0000e+00
Epoch 77/100
153/153 [==============================] - 0s 2ms/step - loss: 8.0311 - precision_37: 0.0000e+00
Epoch 78/100
153/153 [==============================] - 0s 2ms/step - loss: 8.0311 - precision_37: 0.0000e+00
Epoch 79/100
153/153 [==============================] - 0s 2ms/step - loss: 8.0311 - precision_37: 0.0000e+00
Epoch 80/100
153/153 [==============================] - 0s 2ms/step - loss: 8.0311 - precision_37: 0.0000e+00
Epoch 81/100
153/153 [==============================] - 0s 2ms/step - loss: 8.0311 - precision_37: 0.0000e+00
Epoch 82/100
153/153 [==============================] - 0s 2ms/step - loss: 8.0311 - precision_37: 0.0000e+00
Epoch 83/100
153/153 [==============================] - 0s 2ms/step - loss: 8.0311 - precision_37: 0.0000e+00
Epoch 84/100
153/153 [=

...

c:\users\caiol\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


382/382 [==============================] - 2s 2ms/step - loss: 7.8642 - precision_40: 0.0000e+00
Epoch 2/100
382/382 [==============================] - 1s 2ms/step - loss: 7.8642 - precision_40: 0.0000e+00
Epoch 3/100
382/382 [==============================] - 1s 2ms/step - loss: 7.8643 - precision_40: 0.0000e+00
Epoch 4/100
382/382 [==============================] - 1s 2ms/step - loss: 7.8642 - precision_40: 0.0000e+00
Epoch 5/100
382/382 [==============================] - 1s 2ms/step - loss: 7.8643 - precision_40: 0.0000e+00
Epoch 6/100
382/382 [==============================] - 1s 2ms/step - loss: 7.8642 - precision_40: 0.0000e+00
Epoch 7/100
382/382 [==============================] - 1s 2ms/step - loss: 7.8642 - precision_40: 0.0000e+00
Epoch 8/100
382/382 [==============================] - 1s 2ms/step - loss: 7.8643 - precision_40: 0.0000e+00
Epoch 9/100
382/382 [==============================] - 1s 2ms/step - loss: 7.8643 - precision_40: 0.0000e+00
Epoch 10/100
382/382 [=========

382/382 [==============================] - 1s 2ms/step - loss: 7.8643 - precision_40: 0.0000e+00
Epoch 76/100
382/382 [==============================] - 1s 2ms/step - loss: 7.8642 - precision_40: 0.0000e+00
Epoch 77/100
382/382 [==============================] - 1s 2ms/step - loss: 7.8643 - precision_40: 0.0000e+00
Epoch 78/100
382/382 [==============================] - 1s 2ms/step - loss: 7.8642 - precision_40: 0.0000e+00
Epoch 79/100
382/382 [==============================] - 1s 2ms/step - loss: 7.8642 - precision_40: 0.0000e+00
Epoch 80/100
382/382 [==============================] - 1s 2ms/step - loss: 7.8643 - precision_40: 0.0000e+00
Epoch 81/100
382/382 [==============================] - 1s 3ms/step - loss: 7.8642 - precision_40: 0.0000e+00
Epoch 82/100
382/382 [==============================] - 1s 3ms/step - loss: 7.8642 - precision_40: 0.0000e+00
Epoch 83/100
382/382 [==============================] - 1s 2ms/step - loss: 7.8642 - precision_40: 0.0000e+00
Epoch 84/100
382/382 [=

c:\users\caiol\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


ValueError: in user code:

    File "C:\Users\caiol\AppData\Roaming\Python\Python37\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\caiol\AppData\Roaming\Python\Python37\site-packages\keras\engine\training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\caiol\AppData\Roaming\Python\Python37\site-packages\keras\engine\training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\caiol\AppData\Roaming\Python\Python37\site-packages\keras\engine\training.py", line 894, in train_step
        return self.compute_metrics(x, y, y_pred, sample_weight)
    File "C:\Users\caiol\AppData\Roaming\Python\Python37\site-packages\keras\engine\training.py", line 987, in compute_metrics
        self.compiled_metrics.update_state(y, y_pred, sample_weight)
    File "C:\Users\caiol\AppData\Roaming\Python

In [40]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(train_series[predictors].shape[1:])))
model.add(Dense(units=10, activation='relu', input_shape=(train_series[predictors].shape[1:])))
model.add(Dense(units=6, activation='relu', input_shape=(train_series[predictors].shape[1:])))
model.add(Dense(1, activation = 'sigmoid'))

In [41]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', Precision()])

In [42]:
model.fit(train_series[predictors], train_series['target'], batch_size=10, epochs=5)

Epoch 1/5
458/458 [==============================] - 2s 2ms/step - loss: 33123.7891 - accuracy: 0.5030 - precision_11: 0.5139
Epoch 2/5
458/458 [==============================] - 1s 2ms/step - loss: 24990.6758 - accuracy: 0.4949 - precision_11: 0.5062
Epoch 3/5
458/458 [==============================] - 1s 2ms/step - loss: 24177.2891 - accuracy: 0.5078 - precision_11: 0.5188
Epoch 4/5
458/458 [==============================] - 1s 2ms/step - loss: 22155.0078 - accuracy: 0.5106 - precision_11: 0.5214
Epoch 5/5
458/458 [==============================] - 1s 2ms/step - loss: 22500.5430 - accuracy: 0.4977 - precision_11: 0.5090


## Evaluating the model

In [43]:
model.evaluate(test_series[predictors], test_series['target'])

16/16 [==============================] - 1s 2ms/step - loss: 11111.2568 - accuracy: 0.5152 - precision_11: 0.5152


[11111.2568359375, 0.5151515007019043, 0.5151515007019043]

In [44]:
preds = model.predict(test_series[predictors])
preds = preds.reshape(-1)
preds = pd.Series(preds, index=test_series.index)

16/16 [==============================] - 0s 2ms/step


In [45]:
preds

Date
2020-08-04    1.0
2020-08-05    1.0
2020-08-06    1.0
2020-08-07    1.0
2020-08-10    1.0
             ... 
2022-07-26    1.0
2022-07-27    1.0
2022-07-28    1.0
2022-07-29    1.0
2022-08-01    1.0
Length: 495, dtype: float32

In [46]:
preds[preds >= 0.5] = 1
preds[preds < 0.5] = 0

In [47]:
preds

Date
2020-08-04    1.0
2020-08-05    1.0
2020-08-06    1.0
2020-08-07    1.0
2020-08-10    1.0
             ... 
2022-07-26    1.0
2022-07-27    1.0
2022-07-28    1.0
2022-07-29    1.0
2022-08-01    1.0
Length: 495, dtype: float32

In [62]:
precision_score(test_series['target'], preds)

0.5151515151515151

In [63]:
recall_score(test_series['target'], preds)

1.0

In [64]:
accuracy_score(test_series['target'], preds)

0.5151515151515151

In [65]:
confusion_matrix(test_series['target'], preds)

array([[  0, 240],
       [  0, 255]], dtype=int64)